In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
import os
import random
import numpy as np
import subprocess

In [3]:
Session_Name = 'side_view_inpaint_400'

%cd '/home/ubuntu/dreambooth/models/'
!mkdir $Session_Name

OUTPUT_DIR = '/home/ubuntu/dreambooth/models/' + Session_Name + '/'

%cd '/home/ubuntu/dreambooth/workspace'
!mkdir $Session_Name

SESSION_DIR = '/home/ubuntu/dreambooth/workspace/' + Session_Name

%cd $SESSION_DIR
!mkdir instance_images
!mkdir captions

INSTANCE_DIR = SESSION_DIR +'/instance_images'
HOME_DIR = '/home/ubuntu/dreambooth/dreambooth_inpaining/'
CAPTIONS_DIR = SESSION_DIR + "/captions"

%cd {HOME_DIR}

/home/ubuntu/dreambooth/models
/home/ubuntu/dreambooth/workspace
/home/ubuntu/dreambooth/workspace/side_view_inpaint_400
/home/ubuntu/dreambooth/dreambooth_inpaining


In [11]:
seed = None # seed value for reproducible training
if seed is None or seed == 0:
    seed = random.randint(1, 999999)
else:
    seed = int(seed)

# pretrained model name or path to be used for training
pretrained_model_name_or_path = "runwayml/stable-diffusion-inpainting" 

# path to the directory containing the instance images for training
instance_data_dir = INSTANCE_DIR 

# directory for saving model predictions and checkpoints
output_dir = OUTPUT_DIR 

# directory for session data
session_dir = SESSION_DIR 

# directory for captions
captions_dir = CAPTIONS_DIR

# number of steps to train the text encoder ----------------CHANGE----------------
stop_text_encoder_training = 400 

# maximum number of training steps ----------------CHANGE----------------
max_train_steps = 3000

# save the model every n global_steps ----------------CHANGE----------------
save_n_steps = 200 

# filename for image captions
image_captions_filename = True 

# flag to only train the U-Net
train_only_unet = True 

# starting step at which to save the model
save_starting_step = save_n_steps 

# resolution for input images
resolution = 512 

# precision level for training
mixed_precision = "fp16" 

# batch size for training
train_batch_size = 1 

# number of gradient accumulation steps
gradient_accumulation_steps = 1 

# flag to enable gradient checkpointing
gradient_checkpointing = True 

# learning rate for training
learning_rate = 1e-05 

# learning rate scheduler to use
lr_scheduler = "polynomial" 

# number of warmup steps for the learning rate scheduler
lr_warmup_steps = 0 

# prompt identifying the instance images
instance_prompt = "" 

# whether training should be resumed from a previous checkpoint. Use a path saved by `--checkpointing_steps`, or `"latest"` to automatically select the last available checkpoint.
resume_from_checkpoint = None 



#tokenizer_name = None # tokenizer name or path to be used with the model
#class_data_dir = None # path to the directory containing the class images for training, if applicable
#class_prompt = "" # prompt identifying the class images, if applicable
#with_prior_preservation = False # flag to include prior preservation loss in training
#prior_loss_weight = 1.0 # weight of the prior preservation loss
#num_class_images = 100 # minimum number of class images for prior preservation loss
#center_crop = False # flag to center crop images before resizing to resolution

def train(pretrained_model_name_or_path, stop_text_encoder_training, max_train_steps, resume_from_checkpoint):
    
    image_captions_filename_arg = ''
    if image_captions_filename:
        image_captions_filename_arg = '--image_captions_filename'
    
    gradient_checkpointing_arg = ''
    if gradient_checkpointing:
        gradient_checkpointing_arg = '--gradient_checkpointing'
        
    train_only_unet_arg = ''
    if train_only_unet:
        train_only_unet_arg = '--train_only_unet'
        
    resume_from_checkpoint_arg = ''
    if resume_from_checkpoint:
        resume_from_checkpoint_arg = '--resume_from_checkpoint="latest"'

    
    !accelerate launch train_dreambooth_inpaint_V2.py \
        {image_captions_filename_arg} \
        {gradient_checkpointing_arg} \
        {train_only_unet_arg} \
        {resume_from_checkpoint_arg} \
        --pretrained_model_name_or_path="{pretrained_model_name_or_path}" \
        --instance_data_dir="{instance_data_dir}" \
        --instance_prompt="{instance_prompt}" \
        --output_dir="{output_dir}" \
        --home_dir="{HOME_DIR}" \
        --seed={seed} \
        --resolution={resolution} \
        --stop_text_encoder_training={stop_text_encoder_training} \
        --save_starting_step={save_starting_step} \
        --save_n_steps={save_n_steps} \
        --Session_dir="{session_dir}" \
        --captions_dir="{captions_dir}" \
        --mixed_precision="{mixed_precision}" \
        --train_batch_size={train_batch_size} \
        --gradient_accumulation_steps={gradient_accumulation_steps} \
        --learning_rate={learning_rate} \
        --lr_scheduler="{lr_scheduler}" \
        --lr_warmup_steps={lr_warmup_steps} \
        --max_train_steps={max_train_steps}
    
    return True
    

def save(file_name):
    
    ckpt_path = SESSION_DIR + "/"  + Session_Name + '-' + file_name + "-fp16-inpainting.ckpt"

    fp16 = True
    half_arg = ''
    if fp16:
        half_arg = "--half"
    
    #text = f'''
    !python convert_diffusers_to_original_stable_diffusion.py --model_path $OUTPUT_DIR  --checkpoint_path $ckpt_path $half_arg
    #'''
    #print(text)
    
    print(f"[*] Converted ckpt saved at {ckpt_path}")

In [12]:
#!wget https://raw.githubusercontent.com/V4-69/dreambooth_inpainting_c/main/train_dreambooth_inpaint_V2.py

#!cp -R -T '/home/ubuntu/dreambooth/data/side_view_1/instance_images' $INSTANCE_DIR
#!cp -R -T '/home/ubuntu/dreambooth/data/side_view_1/captions' $CAPTIONS_DIR

In [ ]:
%cd {HOME_DIR}
res = train(pretrained_model_name_or_path, stop_text_encoder_training, max_train_steps, resume_from_checkpoint)
file_name = str(max_train_steps)
save(file_name)

/home/ubuntu/dreambooth/dreambooth_inpaining
The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
/home/ubuntu/anaconda/lib/python3.9/site-packages/diffusers/utils/deprecation_utils.py:35: FutureWarning: It is deprecated to pass a pretrained model name or path to `from_config`.If you were trying to load a scheduler, please use <class 'diffusers.schedulers.scheduling_ddpm.DDPMScheduler'>.from_pretrained(...) instead. Otherwise, please make sure to pass a configuration dictionary instead. This functionality will be removed in v1.0.0.
  warnings.warn(warning + message, FutureWarning)
Progress:|                         |:   2%| | 65/3000 [00:48<29:57,  1.63it/s, l s

In [ ]:
# %cd {OUTPUT_DIR}
# !rm -rf ./*

In [ ]:
%cd {HOME_DIR}

In [ ]:
sideviewinp7

In [ ]:
%cd '/home/ubuntu/dreambooth/models/sideviewinp8'

In [6]:
#!rm -rf ./*

In [7]:
ls

In [8]:
#%cd '/home/ubuntu/.local/share/Trash/files'

/home/ubuntu/.local/share/Trash/files


In [9]:
ls

In [ ]:
args.Session_dir[-args.Session_dir[::-1].find('/'):] + '_step_20' + '_fp16_' 

In [ ]:
session_dir[-session_dir[::-1].find('/'):] + '_step_20' + '_fp16_' 

In [ ]:
save_dir = '/home/ubuntu/dreambooth/models/side_view_inpaint/_step_20'
chkpth = '/home/ubuntu/dreambooth/workspace/side_view_inpaint/_step_20.ckpt'
home_dir = HOME_DIR

In [ ]:
subprocess.call(f'python {home_dir}/convert_diffusers_to_original_stable_diffusion.py --model_path {save_dir} --checkpoint_path {chkpth} --half', shell=True)

In [ ]:
chkpth

In [ ]:
output_dir

In [ ]:
save_dir

In [ ]:
session_dir

In [ ]:
output_dir

In [ ]:
HOME_DIR

In [ ]:
image_captions_filename_arg = ''
if image_captions_filename:
    image_captions_filename_arg = '--image_captions_filename'

gradient_checkpointing_arg = ''
if gradient_checkpointing:
    gradient_checkpointing_arg = '--gradient_checkpointing'

train_only_unet_arg = ''
if train_only_unet:
    train_only_unet_arg = '--train_only_unet'

command = f'''
accelerate launch train_dreambooth_inpaint_V2.py \
    {image_captions_filename_arg} \
    {gradient_checkpointing_arg} \
    {train_only_unet_arg} \
    --pretrained_model_name_or_path="{pretrained_model_name_or_path}" \
    --instance_data_dir="{instance_data_dir}" \
    --instance_prompt="{instance_prompt}" \
    --output_dir="{output_dir}" \
    --seed={seed} \
    --resolution={resolution} \
    --stop_text_encoder_training={stop_text_encoder_training} \
    --save_starting_step={save_starting_step} \
    --save_n_steps={save_n_steps} \
    --session_dir="{session_dir}" \
    --captions_dir="{captions_dir}" \
    --mixed_precision="{mixed_precision}" \
    --train_batch_size={train_batch_size} \
    --gradient_accumulation_steps={gradient_accumulation_steps} \
    --learning_rate={learning_rate} \
    --lr_scheduler="{lr_scheduler}" \
    --lr_warmup_steps={lr_warmup_steps} \
    --max_train_steps={max_train_steps} \
    --resume_from_checkpoint={resume_from_checkpoint}
'''
print(command)

In [ ]:
ls

In [ ]:
ls

In [ ]:
ls

In [ ]:
%cd '/home/ubuntu/dreambooth/models/side_view_inpaint_step_20'